# Database init

In [12]:
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3.2",
)

db = Chroma(
    persist_directory='./db/chroma', 
    embedding_function=embeddings)

## Chain for a QA

In [ ]:
!ollama pull llama3.2

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain.prompts import PromptTemplate
from pprint import pprint

# retriever = db.as_retriever(
#     search_type="similarity",
#     search_kwargs={'k': 10}
# )

# retriever = db.as_retriever(
#     search_type="similarity_score_threshold",
#     search_kwargs={'k': 7, 'score_threshold': 0.4}
# )

retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 10, 'lambda_mult': 0.6}
)


from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2")


#prompt = hub.pull("rlm/rag-prompt")

prompt = PromptTemplate.from_template(
    """Use the following pieces of context to answer the question at the end. If you don't know the answer or if you don't have enough information in the context, just say that you don't know, don't try to make up an answer.
Be very concise and to the point. Don't write more than 2 sentences. Do not start your answer with things like "based on the context" or "I think".

Context:
---------
{context}
---------
Question: {question}
Helpful Answer:""" 
)


def format_docs(docs):
    return "\n\n".join(
        doc.page_content for doc in docs
    )

qa_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["docs"])))
    | prompt
    | llm
    | StrOutputParser()
)

# get the source documents
qa_chain_with_source = RunnableParallel(
    {"docs": retriever, "question": RunnablePassthrough()}
).assign(answer=qa_chain)

def print_answer(input):
    answer = qa_chain_with_source.invoke(input)
    print(answer['answer'])
    print("--------------------------")
    pprint(answer)




question = "Where do Windriders often spend their lives?"
# question = "What does the Skydancer eat?"
# question = "What are the physical differences between the Windrider and the Skydancer?"

print_answer(question)
